# Chuẩn bị thư viện

In [4]:
!pip3 install selenium
!mkdir -p data
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import os
import json


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip3 install --upgrade pip


In [5]:
# selenium setups
## https://www.tutorialspoint.com/selenium/selenium_webdriver_chrome_webdriver_options.htm

chrome_options = webdriver.ChromeOptions()

# chrome_options.add_argument('--headless') # must options for Google Colab
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")


In [6]:
MAGAZINE_NAME = "vietnamnet"
HOME_PAGE = "https://vietnamnet.vn/"

# Thu thập dữ liệu

In [7]:
driver = webdriver.Chrome(options=chrome_options)
# Vào trang web chính, mặc định phải chờ toàn bộ trang webload mới xong
driver.get(HOME_PAGE)

In [8]:
# Lấy hết tất cả thể loại từ đây
cats = []

#### CODE
# row_menu = driver.find_elements(by=By.CLASS_NAME, value="header_submenu-content")
sub_menu = driver.find_element(by=By.CLASS_NAME, value="header_submenu-content-list")

cat_menus = sub_menu.find_elements(by=By.TAG_NAME, value="a")

for cat_name in cat_menus:
    cat = cat_name.get_attribute("title").strip()
    href = cat_name.get_attribute("href").strip()
    
    # bỏ qua logo-premium, logo-htvn
    if cat == "premium vietnamnet" or cat == "Hành trình Việt Nam":
        continue
    else:
        cats.append({"cat_names": cat, "url": href})
####

In [9]:
cats, len(cats)

([{'cat_names': 'Chính trị', 'url': 'https://vietnamnet.vn/chinh-tri'},
  {'cat_names': 'Thời sự', 'url': 'https://vietnamnet.vn/thoi-su'},
  {'cat_names': 'Kinh doanh', 'url': 'https://vietnamnet.vn/kinh-doanh'},
  {'cat_names': 'Dân tộc và Tôn giáo',
   'url': 'https://vietnamnet.vn/dan-toc-ton-giao'},
  {'cat_names': 'Thể thao', 'url': 'https://vietnamnet.vn/the-thao'},
  {'cat_names': 'Giáo dục', 'url': 'https://vietnamnet.vn/giao-duc'},
  {'cat_names': 'Thế giới', 'url': 'https://vietnamnet.vn/the-gioi'},
  {'cat_names': 'Đời sống', 'url': 'https://vietnamnet.vn/doi-song'},
  {'cat_names': 'Văn hóa - Giải trí',
   'url': 'https://vietnamnet.vn/van-hoa-giai-tri'},
  {'cat_names': 'Sức khỏe', 'url': 'https://vietnamnet.vn/suc-khoe'},
  {'cat_names': 'Công nghệ', 'url': 'https://vietnamnet.vn/cong-nghe'},
  {'cat_names': 'Pháp luật', 'url': 'https://vietnamnet.vn/phap-luat'},
  {'cat_names': 'Ô tô xe máy', 'url': 'https://vietnamnet.vn/oto-xe-may'},
  {'cat_names': 'Du lịch', 'url': 

In [10]:
TARGET_CATS = [
    "Chính trị",
    "Thời sự",
    "Kinh doanh",
    "Văn hóa - Giải trí",
    "Sức khỏe",
    "Pháp luật",
    "Thế giới",
    "Công nghệ",
    "Thể thao",
    "Du lịch",
    "Giáo dục"
]

filtered_cats = [c for c in cats if c['cat_names'] in TARGET_CATS]

print(len(filtered_cats))
for c in filtered_cats:
    print(c)


11
{'cat_names': 'Chính trị', 'url': 'https://vietnamnet.vn/chinh-tri'}
{'cat_names': 'Thời sự', 'url': 'https://vietnamnet.vn/thoi-su'}
{'cat_names': 'Kinh doanh', 'url': 'https://vietnamnet.vn/kinh-doanh'}
{'cat_names': 'Thể thao', 'url': 'https://vietnamnet.vn/the-thao'}
{'cat_names': 'Giáo dục', 'url': 'https://vietnamnet.vn/giao-duc'}
{'cat_names': 'Thế giới', 'url': 'https://vietnamnet.vn/the-gioi'}
{'cat_names': 'Văn hóa - Giải trí', 'url': 'https://vietnamnet.vn/van-hoa-giai-tri'}
{'cat_names': 'Sức khỏe', 'url': 'https://vietnamnet.vn/suc-khoe'}
{'cat_names': 'Công nghệ', 'url': 'https://vietnamnet.vn/cong-nghe'}
{'cat_names': 'Pháp luật', 'url': 'https://vietnamnet.vn/phap-luat'}
{'cat_names': 'Du lịch', 'url': 'https://vietnamnet.vn/du-lich'}


# Thu thập URL bài báo

In [11]:
NUM_ARTICLES_PER_CAT = 10 # có thể tăng lên

DATA_URL_FILE = "data/vietnamnet_url.json"

# Bổ sung cài đặt chromedriver
## Ta đặt load stategy ở đây là normal: https://www.selenium.dev/documentation/webdriver/drivers/options/
chrome_options.page_load_strategy = "normal"
driver = webdriver.Chrome(options=chrome_options)

## Thử nghiệm

In [12]:
driver = webdriver.Chrome(options=chrome_options)

In [13]:
filtered_cats[1]['url']

'https://vietnamnet.vn/thoi-su'

In [15]:
driver.get(filtered_cats[1]['url'])

In [16]:
title_news = driver.find_elements(by=By.CLASS_NAME, value="vnn-title")
print(title_news[0].text)

Màn đối đáp 'cực gắt' khiến kẻ giả danh cán bộ thuế nghẹn ngào thú tội


In [17]:
url_new = title_news[0].find_element(by=By.TAG_NAME, value="a").get_attribute("href")
print(url_new)

https://vietnamnet.vn/man-doi-dap-cuc-gat-khien-ke-gia-danh-can-bo-thue-nghen-ngao-thu-toi-2433305.html


In [18]:
# Tìm layout thứ 2 của trang web

#### CODE
page_numbers = driver.find_element(by=By.CLASS_NAME, value="topStory-2nd")
next_page = page_numbers.find_elements(by=By.TAG_NAME, value="a")
# print(len(next_page))
# print(next_page[0].get_attribute("href"))
# print(next_page[1].get_attribute("href"))
# print(next_page[2].get_attribute("href"))
# print(next_page[3].get_attribute("href"))
# print(next_page[4].get_attribute("href"))
# print(next_page[5].get_attribute("href"))
next_page = next_page[3].get_attribute("href")
print(next_page)

https://vietnamnet.vn/du-bao-thoi-tiet-20-8-2025-mien-bac-nang-den-34-do-van-xuat-hien-mua-to-cuc-bo-2433768.html


In [19]:
driver.close()

## Chạy thật

In [20]:
driver = webdriver.Chrome(options=chrome_options)

In [21]:
NUM_ARTICLES_PER_CAT = 3
crawled_urls = set()

def get_avatar_url_by_article_url(driver, article_url):
    try:
        avatar_divs = driver.find_elements(by=By.CLASS_NAME, value="verticalPost__avt")
        
        for avatar_div in avatar_divs:
            try:
                link_element = avatar_div.find_element(by=By.TAG_NAME, value="a")
                href = link_element.get_attribute("href")
                
                # Nếu href match với article_url (có thể là relative hoặc absolute)
                if href and (href == article_url or article_url.endswith(href) or href in article_url):
                    img_element = avatar_div.find_element(by=By.TAG_NAME, value="img")
                    print(f"Found avatar for {article_url}: {img_element.get_attribute('src')}")
                    return img_element.get_attribute("src")
                
            except NoSuchElementException:
                continue
        
        return None
    except Exception as e:
        print(f"Error getting avatar for {article_url}: {e}")
        return None

def crawl_each_category_url(driver, category_url):
    all_data = []
    url = category_url
    
    while len(all_data) < NUM_ARTICLES_PER_CAT:
        driver.get(url)
        title_news = driver.find_elements(by=By.CLASS_NAME, value="vnn-title")
        
        for title in title_news:
            try:
                url_new = title.find_element(by=By.TAG_NAME, value="a").get_attribute("href")
                if url_new.startswith(HOME_PAGE) and url_new not in crawled_urls:
                    avatar_url = get_avatar_url_by_article_url(driver, url_new)
                    
                    article_data = {
                        "url": url_new,
                        "url_img": avatar_url
                    }
                    
                    all_data.append(article_data)
                    crawled_urls.add(url_new)

                    if len(all_data) >= NUM_ARTICLES_PER_CAT:
                        break
                        
            except (StaleElementReferenceException, NoSuchElementException):
                continue

            except Exception as e:
                print(f"Error processing article: {e}")
        
        if len(all_data) >= NUM_ARTICLES_PER_CAT:
            break
        
        try:
            next_pages = driver.find_element(by=By.CLASS_NAME, value="pagination__list")
            url = next_pages.find_elements(by=By.TAG_NAME, value="a")[6].get_attribute("href")
        except (NoSuchElementException, IndexError):
            print("Cannot find next page, stopping crawl for this category")
            break
    
    return all_data

    

In [22]:
saved_cats = {}
for cat in cats:
    cat_name = cat['cat_names']
    url = cat['url']
    if cat_name in TARGET_CATS:
        print(f"You are at {cat}.")
        article_data = crawl_each_category_url(driver, url)
        saved_cats[cat_name] = article_data


with open(DATA_URL_FILE, "w", encoding="utf-8") as fOut:
    json.dump(saved_cats, fOut, ensure_ascii=False, indent=4)

driver.close()

You are at {'cat_names': 'Chính trị', 'url': 'https://vietnamnet.vn/chinh-tri'}.
Found avatar for https://vietnamnet.vn/quy-dinh-moi-ve-cac-truong-hop-mien-giam-tien-su-dung-dat-tien-thue-dat-2433921.html: https://static-images.vnncdn.net/vps_images_publish/000001/000003/2025/8/20/quy-dinh-moi-ve-cac-truong-hop-mien-giam-tien-su-dung-dat-tien-thue-dat-549.jpeg?width=550&s=r7mXsWGDUz1nFREstkCRWQ
Found avatar for https://vietnamnet.vn/bo-quoc-phong-de-xuat-thanh-lap-ban-chi-dao-phong-khong-nhan-dan-trung-uong-2433796.html: https://static-images.vnncdn.net/vps_images_publish/000001/000003/2025/8/19/bo-quoc-phong-de-xuat-thanh-lap-ban-chi-dao-phong-khong-nhan-dan-trung-uong-4199.jpg?width=360&s=98Bu80MT1Lrmp2Jq95SM-w
Found avatar for https://vietnamnet.vn/phu-nhan-chu-tich-nuoc-va-hoang-hau-bhutan-van-canh-chua-tran-quoc-duoi-mua-2433621.html: https://static-images.vnncdn.net/vps_images_publish/000001/000003/2025/8/19/phu-nhan-chu-tich-nuoc-va-hoang-hau-bhutan-van-canh-chua-tran-quoc-duoi-

In [23]:
len(crawled_urls)

33

# Thu thập thông tin bài báo

In [24]:
# Filepath cho cái trước
FILE_URL_PATH = "data/vietnamnet_url.json"

# Đặt limit số articles lấy từ mỗi thể loại
MAX_ARTICLES_PER_CAT = 2 # Nếu set = None thì là tất cả các urls ở file trước

# Data output, mỗi thể loại là 1 file json chứa articles
DATA_FOLDER_OUTPUT = "data\\vietnamnet"

!mkdir -p $DATA_FOLDER_OUTPUT

# Để loading strategy về eager load nhanh, không quan tâm ảnh
chrome_options.page_load_strategy = "eager"

In [25]:
# Đọc file url

with open(FILE_URL_PATH, "r", encoding="utf-8") as fIn:
    url_data = json.load(fIn)

len(url_data)

11

## Chạy thử nghiệm

In [67]:
driver = webdriver.Chrome(options=chrome_options)


In [68]:

# Một số url để thử nghiệm

SAMPLE_ARTICLE_URLS = [
        {
            "url": "https://vietnamnet.vn/man-doi-dap-cuc-gat-khien-ke-gia-danh-can-bo-thue-nghen-ngao-thu-toi-2433305.html",
            "url_img": "https://static-images.vnncdn.net/vps_images_publish/000001/000003/2025/8/19/man-doi-dap-cuc-gat-khien-ke-gia-danh-can-bo-thue-nghen-ngao-thu-toi-1546.gif?width=550&s=o2b7BQcMjEroZaFOnitZTw"
        },
        {
            "url": "https://vietnamnet.vn/nguy-co-rinh-rap-tu-diem-den-loi-di-tu-mo-qua-duong-sat-o-ninh-binh-2432916.html",
            "url_img": "https://static-images.vnncdn.net/vps_images_publish/000001/000003/2025/8/17/hanh-dong-lieu-linh-cua-nguoi-dan-ninh-binh-khi-song-gan-duong-ray-tau-hoa-998.jpg?width=360&s=Xo-IcBTRm58t6H6s-S8iPg"
        },
        {
            "url": "https://vietnamnet.vn/du-bao-thoi-tiet-20-8-2025-mien-bac-nang-den-34-do-van-xuat-hien-mua-to-cuc-bo-2433768.html",
            "url_img": "https://static-images.vnncdn.net/vps_images_publish/000001/000003/2025/8/20/2025-mien-bac-nang-den-34-do-van-xuat-hien-mua-to-cuc-bo-255.jpg?width=360&s=R4M4D3fBLPTkOTsLldQ3mA"
        }
]

# chọn url
SAMPLE_ARTICLE_URL = SAMPLE_ARTICLE_URLS[2]

In [70]:
import json
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

def get_content_metadata(driver, article_url, url_img=None):
    """
    Extracts and returns metadata and content from a given article URL.

    :param driver: Selenium WebDriver instance.
    :param article_url: URL of the article to extract data from.
    :param url_img: (optional) URL of the article's thumbnail/cover image.
    :return: Dictionary containing article metadata and content.
    """

    # Get to current article
    driver.get(article_url)

    # Title & description
    title = driver.find_element(By.CSS_SELECTOR, "h1.content-detail-title").text.strip()
    description = driver.find_element(By.CLASS_NAME, "sm-sapo-mb-0").text.strip()

    # ---- Categories
    lis_cat = []
    try:
        cats = driver.find_element(By.CLASS_NAME, "sm-show-time")
        lis_cats = cats.find_elements(By.TAG_NAME, "li")
        lis_cats = lis_cats[1:]
        for cat in lis_cats:
            category = cat.text.strip()
            if category:
                lis_cat.append(category)
    except NoSuchElementException:
        pass

    main_cat = lis_cat[0] if lis_cat else "Unknown"
    sub_cat_val = lis_cat[-1] if len(lis_cat) > 1 else "None"

    # ---- Published date
    try:
        published_date = driver.find_element(By.CLASS_NAME, "bread-crumb-detail__time").text.strip()
    except NoSuchElementException:
        published_date = ""

    # ---- Content
    contents = []
    author_str = "Unknown"     # final string for author
    article = driver.find_element(By.CLASS_NAME, "main-content")
    children = article.find_elements(By.XPATH, "./*")

    for child in children:
        text = child.text.strip()
        if not text:
            continue

        if child.tag_name.lower() in {"p", "h1", "h2", "h3", "h4"}:
            contents.append(text)

        elif child.tag_name.lower() == "figure":
            # Caption (<=100) bọc trong [] ; dài hơn coi như đoạn nội dung
            if len(text) <= 100:
                contents.append(f"[{text}]")
            else:
                contents.append(text)
        # tables & others are ignored

    # ---- Author(s)
    authors = []
    try:
        wrapper = driver.find_element(By.CLASS_NAME, "article-detail-author-wrapper")
        author_names = wrapper.find_elements(By.CLASS_NAME, "name")
        for a in author_names:
            # ưu tiên <a title="...">, fallback text
            t = (a.find_element(By.CSS_SELECTOR, "a").get_attribute("title") 
                 if len(a.find_elements(By.CSS_SELECTOR, "a")) 
                 else a.text).strip()
            if t:
                authors.append(t)
    except NoSuchElementException:
        pass

    if authors:
        author_str = ", ".join(dict.fromkeys(authors))  # unique & stable order
    else:
        # fallback: cố gắng lấy dòng cuối giống dạng chữ ký tác giả
        if contents:
            last = contents[-1]
            dash_pos = last.find('-')
            if 0 <= dash_pos <= 30:
                # ví dụ: "Nguyễn A - Tổng hợp"
                possible = last[:dash_pos].strip()
                if possible:
                    author_str = possible

    return {
        "url": article_url,
        "url_img": url_img,                # <-- added
        "title": title,
        "description": description,
        "content": "\n".join(contents),
        "metadata": {
            "cat": main_cat,
            "subcat": sub_cat_val,
            "published_date": published_date,
            "author": author_str
        }
    }

# ---- Crawl & save JSON
results = []
for item in SAMPLE_ARTICLE_URLS:
    data = get_content_metadata(driver, item["url"], url_img=item.get("url_img"))
    results.append(data)

with open("articles.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("Đã lưu 3 bài vào articles.json")


Đã lưu 3 bài vào articles.json


In [63]:
driver.close()

## Chạy thật

In [71]:
import json
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

def get_content_metadata(driver, article_url, url_img=None):
    """
    Extracts and returns metadata and content from a given article URL.

    :param driver: Selenium WebDriver instance.
    :param article_url: URL of the article to extract data from.
    :param url_img: (optional) URL of the article's thumbnail/cover image.
    :return: Dictionary containing article metadata and content.
    """

    # Get to current article
    driver.get(article_url)

    # Title & description
    title = driver.find_element(By.CSS_SELECTOR, "h1.content-detail-title").text.strip()
    description = driver.find_element(By.CLASS_NAME, "sm-sapo-mb-0").text.strip()

    # ---- Categories
    lis_cat = []
    try:
        cats = driver.find_element(By.CLASS_NAME, "sm-show-time")
        lis_cats = cats.find_elements(By.TAG_NAME, "li")
        lis_cats = lis_cats[1:]
        for cat in lis_cats:
            category = cat.text.strip()
            if category:
                lis_cat.append(category)
    except NoSuchElementException:
        pass

    main_cat = lis_cat[0] if lis_cat else "Unknown"
    sub_cat_val = lis_cat[-1] if len(lis_cat) > 1 else "None"

    # ---- Published date
    try:
        published_date = driver.find_element(By.CLASS_NAME, "bread-crumb-detail__time").text.strip()
    except NoSuchElementException:
        published_date = ""

    # ---- Content
    contents = []
    author_str = "Unknown"     # final string for author
    article = driver.find_element(By.CLASS_NAME, "main-content")
    children = article.find_elements(By.XPATH, "./*")

    for child in children:
        text = child.text.strip()
        if not text:
            continue

        if child.tag_name.lower() in {"p", "h1", "h2", "h3", "h4"}:
            contents.append(text)

        elif child.tag_name.lower() == "figure":
            # Caption (<=100) bọc trong [] ; dài hơn coi như đoạn nội dung
            if len(text) <= 100:
                contents.append(f"[{text}]")
            else:
                contents.append(text)
        # tables & others are ignored

    # ---- Author(s)
    authors = []
    try:
        wrapper = driver.find_element(By.CLASS_NAME, "article-detail-author-wrapper")
        author_names = wrapper.find_elements(By.CLASS_NAME, "name")
        for a in author_names:
            # ưu tiên <a title="...">, fallback text
            t = (a.find_element(By.CSS_SELECTOR, "a").get_attribute("title") 
                 if len(a.find_elements(By.CSS_SELECTOR, "a")) 
                 else a.text).strip()
            if t:
                authors.append(t)
    except NoSuchElementException:
        pass

    if authors:
        author_str = ", ".join(dict.fromkeys(authors))  # unique & stable order
    else:
        # fallback: cố gắng lấy dòng cuối giống dạng chữ ký tác giả
        if contents:
            last = contents[-1]
            dash_pos = last.find('-')
            if 0 <= dash_pos <= 30:
                # ví dụ: "Nguyễn A - Tổng hợp"
                possible = last[:dash_pos].strip()
                if possible:
                    author_str = possible

    return {
        "url": article_url,
        "url_img": url_img,                # <-- added
        "title": title,
        "description": description,
        "content": "\n".join(contents),
        "metadata": {
            "cat": main_cat,
            "subcat": sub_cat_val,
            "published_date": published_date,
            "author": author_str
        }
    }


In [72]:
FILE_URL_PATH = "data/vietnamnet_url.json"  
MAX_ARTICLES_PER_CAT = None
DATA_FOLDER_OUTPUT = os.path.join("data", "vietnamnet")  # path portable cho mọi OS
os.makedirs(DATA_FOLDER_OUTPUT, exist_ok=True)

In [73]:
with open(FILE_URL_PATH, "r", encoding="utf-8") as fIn:
    url_data = json.load(fIn)

len(url_data)

11

In [74]:
driver = webdriver.Chrome(options=chrome_options)

for cat, urls in url_data.items():
    print(f"Thu thập dữ liệu thể loại {cat} ..")
    count_crawled = 0
    cat_data = []

    for url in urls:
        try:
            # Giữ nguyên cấu trúc cũ, chỉ bổ sung url_img:
            if isinstance(url, dict):
                u = url.get("url")
                url_img = url.get("url_img")
            else:
                u = url
                url_img = None

            article = get_content_metadata(driver, u, url_img=url_img)
            cat_data.append(article)
            count_crawled += 1

            if MAX_ARTICLES_PER_CAT and count_crawled >= MAX_ARTICLES_PER_CAT:
                break

        except StaleElementReferenceException:
            print(f"Bug at url: {url}, with StaleElementReferenceException")
            driver.refresh()
            continue

        except NoSuchElementException:
            print(f"Bug at url: {url}, with NoSuchElementException")
            driver.refresh()
            continue

    # Lưu file JSON cho từng thể loại
    name_file_cat = cat.lower().replace(" ", "-") + ".json"
    out_path = os.path.join(DATA_FOLDER_OUTPUT, name_file_cat)
    with open(out_path, "w", encoding="utf-8") as fOut:
        json.dump(cat_data, fOut, ensure_ascii=False, indent=4)

driver.close()
print("Done.")


Thu thập dữ liệu thể loại Chính trị ..
Thu thập dữ liệu thể loại Thời sự ..
Thu thập dữ liệu thể loại Kinh doanh ..
Thu thập dữ liệu thể loại Thể thao ..
Thu thập dữ liệu thể loại Giáo dục ..
Thu thập dữ liệu thể loại Thế giới ..
Thu thập dữ liệu thể loại Văn hóa - Giải trí ..
Thu thập dữ liệu thể loại Sức khỏe ..
Thu thập dữ liệu thể loại Công nghệ ..
Thu thập dữ liệu thể loại Pháp luật ..
Thu thập dữ liệu thể loại Du lịch ..
Done.
